# Import libraries

In [1]:
import sys
sys.path.append("./legal-radar/")
sys.path = list(set(sys.path))
import os
os.getcwd()
os.chdir('./legal-radar/')
from legal_radar.services.semantic_search_evaluation import SemanticSearchEvaluation
from legal_radar import config
from legal_radar.services.model_registry import embedding_registry
import pandas as pd
import plotly.express as px

hint: Pulling without specifying how to reconcile divergent branches is
hint: discouraged. You can squelch this message by running one of the following
hint: commands sometime before your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 9 (delta 7), reused 9 (delta 7), pack-reused 0
Unpacking objects: 100% (9/9), 2.34 KiB | 599.00 KiB/s, done.
From https://github.com/meaningfy-ws/legal-radar
   ab2d03d..0d9f313  master     -> origin/master
Updat

2021-11-26 11:41:40.994161: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-26 11:41:40.994188: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Define constants

In [2]:
FAISS_BUCKET_NAME = 'faiss-index'
FAISS_INDEX_FINREG_NAME = 'faiss_index_finreg'
FIN_REG_SPLITTED_ES_INDEX = 'ds_finreg_splitted'
SAMPLE_QUESTIONS_CSV_PATH = 'tests/test_data/sample_questions_v4.csv'
SAMPLE_QUESTIONS_CSV_SEP = ';'
RESULT_COLUMNS = [
    'in_top_5_slices',	'in_top_10_slices',
    'in_top_5_documents',	'in_top_10_documents',	'in_q3'
]
EXPERIMENT_CONFIGS = [
    (10, 5),
    (20, 10),
    (50, 25),
    (100, 50)
]

# Define evaluation process

In [3]:
def compute_evaluation():
    results = []
    for split_window_size, split_window_step in EXPERIMENT_CONFIGS:
        fin_reg_es_index_name = '_'.join(
            map(str, (FIN_REG_SPLITTED_ES_INDEX, split_window_size, split_window_step)))
        faiss_index_finreg_name = '_'.join(
            map(str, (FAISS_INDEX_FINREG_NAME, split_window_size, split_window_step, '.pkl')))
        semantic_search_evaluation = SemanticSearchEvaluation(documents_es_index_name=config.EU_FINREG_CELLAR_ELASTIC_SEARCH_INDEX_NAME,
                                                              splitted_documents_es_index_name=fin_reg_es_index_name,
                                                              faiss_bucket_name=FAISS_BUCKET_NAME,
                                                              faiss_index_path=faiss_index_finreg_name,
                                                              embedding_model=embedding_registry.sent2vec_universal_sent_encoding(),
                                                              sample_questions_csv_path=SAMPLE_QUESTIONS_CSV_PATH,
                                                              sample_questions_csv_sep=SAMPLE_QUESTIONS_CSV_SEP)
        results.append(semantic_search_evaluation.evaluate())
        del semantic_search_evaluation
    result_df = []
    for index in range(0, len(EXPERIMENT_CONFIGS)):
        for result_columns in RESULT_COLUMNS:
            tmp = results[index][result_columns].value_counts(normalize=True)
            tmp_dict2 = {str(index): tmp[index] for index in tmp.index}
            tmp_dict1 = {
                "split_type": str(EXPERIMENT_CONFIGS[index]),
                "result_column": result_columns,
            }
            tmp_dict1.update(tmp_dict2)
            result_df.append(tmp_dict1)
    result_df = pd.DataFrame(result_df).fillna(0)
    return result_df


INFO:absl:Using /tmp/tfhub_modules to cache modules.
2021-11-26 11:41:52.467289: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-26 11:41:52.467313: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-26 11:41:52.467333: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (293e399508a7): /proc/driver/nvidia/version does not exist
2021-11-26 11:41:52.467470: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100% (5791 of 5791) |####################| Elapsed Time: 

None type!
None type!


100% (5791 of 5791) |####################| Elapsed Time: 0:00:04 Time:  0:00:04
100% (37910 of 37910) |##################| Elapsed Time: 0:00:17 Time:  0:00:17


None type!
None type!
None type!
None type!


100% (5791 of 5791) |####################| Elapsed Time: 0:00:04 Time:  0:00:04
100% (19625 of 19625) |##################| Elapsed Time: 0:00:11 Time:  0:00:11


None type!
None type!
None type!
None type!
None type!
None type!


# Semantic search evaluation

In [ ]:
evaluation_df = compute_evaluation()
fig = px.bar(evaluation_df, x="split_type", y="True", color="result_column",barmode='group', title="Search results based on window size")
fig.update_yaxes(range=[0, 1])
fig.show()
fig = px.bar(evaluation_df, x="split_type", y="False", color="result_column",barmode='group', title="Search results based on window size")
fig.show()